**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:

from scipy.spatial.distance import pdist, squareform
from collections import Counter
from sklearn.neighbors import KDTree


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self._k = n_neighbors
        self.alg = algorithm

    def fit(self, X, y):
        self.train = np.array(X, dtype=float)
        self.res = np.array(y)
        self.size = self.train.shape[0]
        self.unclasses = np.unique(y)
        self.tree = KDTree(self.train, leaf_size=30)

    def predict(self, X):
        if self.alg == "brute":
            d = squareform(pdist(np.concatenate((self.train, X), axis=0)))
            d = d[self.size:, :self.size]
            d = d.argpartition(self._k)[:, :self._k]
            f = self.res[d]
            ans = []
            for obj in f:
                values, counts = np.unique(obj, return_counts=True)
                ind = np.argmax(counts)
                ans.append(values[ind])
        if self.alg == "kd_tree":
            ans = []
            _, d = self.tree.query(X, k=self._k)
            f = self.res[d]
            for obj in f:
                values, counts = np.unique(obj, return_counts=True)
                ind = np.argmax(counts)
                ans.append(values[ind])
        return ans

    def predict_proba(self, X):
        ans = np.zeros(shape=(np.shape(X)[0], self.unclasses.shape[0]))
        if self.alg == "brute":
            d = squareform(pdist(np.concatenate((self.train, X), axis=0)))
            d = d[self.size:, :self.size]
            d = d.argpartition(self._k)[:, :self._k]
            f = self.res[d]
            for i in range(f.shape[0]):
                values, counts = np.unique(f[i], return_counts=True)
                ans[i, values] = counts / self._k
        if self.alg == "kd_tree":
            _, d = self.tree.query(X, k=self._k)
            f = self.res[d]
            for i in range(f.shape[0]):
                values, counts = np.unique(f[i], return_counts=True)
                ans[i, values] = counts / self._k
        return ans

    def score(self, X, y):
        return np.mean(self.predict(X) == y)


**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

print(my_clf.score(X_test, y_test))
print(clf.score(X_test,y_test))
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

0.9333333333333333
0.9333333333333333


**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [4]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [5]:
%time my_clf.fit(X_train, y_train)

Wall time: 532 µs


In [6]:
%time clf.predict(X_test)

Wall time: 1.19 ms


array([1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 2, 1, 0, 1, 0])

In [7]:
%time my_clf.predict(X_test)

Wall time: 885 µs


[1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 2, 1, 0, 1, 0]

In [8]:
%time clf.predict_proba(X_test)

Wall time: 696 µs


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
%time my_clf.predict_proba(X_test)

Wall time: 997 µs


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [10]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [12]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [13]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [14]:
%time clf.predict(X_test)

Wall time: 1 ms


array([2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 2, 2, 2, 1, 0])

In [15]:
%time my_clf.predict(X_test)

Wall time: 998 µs


[2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 2, 2, 2, 1, 0]

In [16]:
%time clf.predict_proba(X_test)

Wall time: 698 µs


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [17]:
%time my_clf.predict_proba(X_test)

Wall time: 997 µs


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [18]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [19]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [20]:
data = newsgroups['data']
target = newsgroups['target']              

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [21]:
data_tok = [text.lower() for text in data]
n = len(data_tok)
i = 0
while i < n:
    data_tok[i] = ''.join([c if c.isalpha() or c.isdigit() else ' ' for c in data_tok[i]])
    data_tok[i] = data_tok[i].split()
    if data_tok[i] == []:
        del data_tok[i]
        target = np.delete(target, i)
        n -= 1
    else:
        i += 1


In [22]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [23]:
from collections import defaultdict


dic_count = defaultdict(int)
for text in data_tok:
    for word in text:
        dic_count[word] += 1


In [24]:
temp_list = list(dic_count.items())
for (word, value) in temp_list:
    if value <= 9:
        del dic_count[word]
    if value >= 2500:
        del dic_count[word]


In [25]:
from scipy.sparse import csr_matrix


temp_list = list(dic_count.items())
indptr = [0]
indices = []
data = []
for d in data_tok:
    for term in d:
        if term not in dic_count:
            continue
        index = temp_list.index((term, dic_count[term]))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

matrix = csr_matrix((data, indices, indptr), dtype=int).toarray()


*Так мы получили векторное предст авление наших текстов. Значит можно приступать к задаче обучения модели*w

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [26]:
from sklearn.model_selection import KFold


kf = KFold(n_splits=3)
for train_index, test_index in kf.split(matrix):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = matrix[train_index], matrix[test_index]
    y_train, y_test = target[train_index], target[test_index]


TRAIN: [ 3668  3669  3670 ... 11001 11002 11003] TEST: [   0    1    2 ... 3665 3666 3667]
TRAIN: [    0     1     2 ... 11001 11002 11003] TEST: [3668 3669 3670 ... 7333 7334 7335]
TRAIN: [   0    1    2 ... 7333 7334 7335] TEST: [ 7336  7337  7338 ... 11001 11002 11003]


Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [27]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.